In [4]:
from statsmodels.tsa.stattools import adfuller, kpss
import pandas as pd

In [5]:
def adf_test(timeseries):
    print("Results of Dickey-Fuller Test:")
    dftest = adfuller(timeseries, autolag="AIC")
    dfoutput = pd.Series(
        dftest[0:4],
        index=[
            "Test Statistic",
            "p-value",
            "#Lags Used",
            "Number of Observations Used",
        ],
    )
    for key, value in dftest[4].items():
        dfoutput[f"Critical Value ({key})"] = value
    print('Série estacionária se p-value < 0.05')
    print('====================================')
    print(dfoutput)

In [6]:
def kpss_test(x, h0_type='c'):
  indices = ['Estatística do teste', 'valor-p', '# de Lags']
  kpss_test = kpss(x, regression=h0_type, nlags="legacy")
  results = pd.Series(kpss_test[0:3], index=indices)
  for key, value in kpss_test[3].items():
    results[f'Critical Value ({key})'] = value
  print('Série estacionária se p-value > 0.05')
  print('====================================')
  return results

In [ ]:
from scipy.signal import hilbert
import numpy as np
def fases_hilbert_transform(time_series):
  ang = np.angle(hilbert(time_series, axis=0))
  return np.mod(ang, 2*np.pi) #retorna ângulo entre 0 e 2pi

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

def plot_picos_acf_pacf(data):
    # Calcular a função de autocorrelação
    fig, ax = plt.subplots(2, 1, figsize=(12, 8))

    # Plotar a função de autocorrelação (ACF)
    plot_acf(data, lags=300, ax=ax[0])
    ax[0].set_title('Função de Autocorrelação')

    # Plotar a função de autocorrelação parcial (PACF)
    plot_pacf(data, lags=300, ax=ax[1])
    ax[1].set_title('Função de Autocorrelação Parcial')
    plt.show()

    # Calcular os valores da autocorrelação e os intervalos de confiança
    acf_values = ax[0].lines[0].get_ydata()
    conf_intervals = np.concatenate([path.vertices[:, 1] for path in ax[0].collections[1].get_paths()])

    # Identificar picos na ACF
    acf_peaks = np.where(acf_values > conf_intervals)[0]
    print("Picos na ACF:", acf_peaks)

    # Identificar picos na PACF
    pacf_values, _ = ax[1].lines[0].get_ydata(), ax[1].collections[1].get_segments()[0][:, 1]
    pacf_peaks = np.where(pacf_values > 0.1)[0]  # Ajuste o limiar conforme necessário
    print("Picos na PACF:", pacf_peaks)

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

# Função para remover tendência das séries
def remove_trend(serie, tipo):
  if tipo == 1:
    decomp = seasonal_decompose(serie, model='additive',period=252)
    result = decomp.seasonal + decomp.resid
  elif tipo == 2:
    decomp = seasonal_decompose(serie, model='multiplicative',period=252)
    result = decomp.seasonal * decomp.resid
  else:
    print("Tipo inválido")
  return result